<a href="https://colab.research.google.com/github/wayne540500/CNN_PyTorch_MNIST_IMAGE/blob/main/CNN_PyTorch_MNIST_IMAGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# convert mnist image into a tensor of 4-dimensions (num of images, height, weigth, color channel)
transform = transforms.ToTensor()

In [3]:
# train data
train_data = datasets.MNIST(root = '/CNN_DATA', train=True, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 112421782.67it/s]


Extracting /CNN_DATA/MNIST/raw/train-images-idx3-ubyte.gz to /CNN_DATA/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 38987992.86it/s]


Extracting /CNN_DATA/MNIST/raw/train-labels-idx1-ubyte.gz to /CNN_DATA/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 34284439.38it/s]


Extracting /CNN_DATA/MNIST/raw/t10k-images-idx3-ubyte.gz to /CNN_DATA/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6386365.66it/s]


Extracting /CNN_DATA/MNIST/raw/t10k-labels-idx1-ubyte.gz to /CNN_DATA/MNIST/raw



In [4]:
# test data
test_data = datasets.MNIST(root = '/CNN_DATA', train=False, download=True, transform=transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /CNN_DATA
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
# create small batch size of image
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [7]:
# define cnn model
# describe cnn and what its doing (2 layer)
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)



In [9]:
# grab 1 MNIST image
for i , (X_Train, y_train) in enumerate(train_data):
  break

In [11]:
X_Train.shape

torch.Size([1, 28, 28])

In [12]:
x = X_Train.view(1,1,28,28)

In [13]:
# perform first cnn
x = F.relu(conv1(x))

In [14]:
x.shape

torch.Size([1, 6, 26, 26])